In [2]:
import requests

url = "https://social.cyware.com/api-social/all-alerts?category_slug=virus-threat-actors-news&l_time=1729161051&page_size=1000"

payload = {}
headers = {
  'accept': 'application/json, text/plain, */*',
  'accept-language': 'en-US,en;q=0.9,fr-FR;q=0.8,fr;q=0.7',
  'cookie': 'hubspotutk=f9925edb2d6b1c2a4773e67f314ce72a; __hssrc=1; _hjSessionUser_2971245=eyJpZCI6ImZkMDFhYzQ0LTkwNzItNTgwNy1hYjQxLWY0ZTVjNjUyM2NlYyIsImNyZWF0ZWQiOjE3MjExMzc1MTQzMzMsImV4aXN0aW5nIjp0cnVlfQ==; auth.strategy=local; _gcl_au=1.1.1944780078.1730907099; _gid=GA1.2.216401688.1730907100; _ga=GA1.1.565507991.1721137515; _gd_svisitor=f5fbce17850b2d00ce4107650d0000005afe1800; _gd_visitor=bf337e49-0f12-47e8-8b12-9141002c0122; _gd_session=9b387e54-e56a-4c8f-8831-35859b8b4987; _hjSession_2971245=eyJpZCI6IjEyNDBmOTc5LWI3ZjktNGI0OC05ZTNkLTdmNDg3YjYyZmMzZCIsImMiOjE3MzA5MDcxMDUyNjAsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; _an_uid=3634267898483236032; _fbp=fb.1.1730907105503.401986166780599856; drift_campaign_refresh=15d81c5a-c3f1-406d-a02c-dcd5643decc2; __hstc=86885063.f9925edb2d6b1c2a4773e67f314ce72a.1721137516728.1721382671764.1730907106653.3; __hssc=86885063.1.1730907106653; drift_aid=bd14cc65-0cc2-479b-8867-74cb44d224e3; driftt_aid=bd14cc65-0cc2-479b-8867-74cb44d224e3; _ga_WPEVCWB5J2=GS1.1.1730907104.3.0.1730907113.51.0.0',
  'priority': 'u=1, i',
  'referer': 'https://social.cyware.com/category/virus-threat-actors-news',
  'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-origin',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.json)


<bound method Response.json of <Response [200]>>


In [10]:
response.json()["results"][1]

{'text': 'Trend Micro researchers are monitoring a cyber espionage group known as Earth Simnavaz, also known as APT34 and OilRig, targeting governmental entities in the UAE and the Gulf region. The group uses sophisticated tactics like leveraging Microsoft Exchange servers for credentials theft and exploiting vulnerabilities for privilege escalation. They blend malicious activity with normal network traffic to evade detection.\n\nRecent activity indicates Earth Simnavaz is focusing on exploiting vulnerabilities in critical infrastructure of geopolitically sensitive regions and establishing a foothold in compromised entities for launching attacks on additional targets. The group primarily targets organizations in the energy sector, particularly oil and gas, using advanced tactics to gain unauthorized access and exfiltrate sensitive information. There has been a notable increase in cyberattacks on governmental sectors in the UAE and the Gulf region, highlighting the ongoing threat posed 

In [16]:
taa_texts = [f'{element["text"]}"\n"{element["desc1"] if element["desc1"]!=None else ""}"\n"{element["desc2"] if element["desc2"]!=None else ""}' for element in response.json()["results"]]

In [17]:
taa_texts

['Water Makara, a threat actor group, has been using obfuscated JavaScript in a spear phishing campaign to target enterprises in Brazil with the Astaroth malware. The phishing emails are disguised as official tax documents to lure victims into downloading the malware. Manufacturing companies, retail firms, and government agencies in Brazil are the primary targets of this campaign.\n\nThe malicious emails contain ZIP files with harmful attachments that use mshta. exe to execute obfuscated JavaScript commands and establish connections to a command and control (C&C) server. Water Makara\'s attack chain involves impersonating legitimate organizations to trick users into opening malicious attachments.\n\nThe ZIP files typically contain LNK files with hidden obfuscated JavaScript commands, which, when executed, run malicious code. Water Makara uses various file extensions such as .pdf, .jpg, .png, .gif, .mov, and .mp4 to spread malware and evade detection.\n\nThe encoded JavaScript commands 

In [3]:
import csv
import os
import re

# Assuming response.json()["results"] contains your actual data
response_results = response.json()["results"]  # This should be your actual response object

# Check where the script is running and ensure we're writing to the correct directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Define the file path explicitly (it will be saved in the current directory)
csv_path = os.path.join(current_dir, 'output.csv')

# Function to remove HTML tags and unwanted newlines
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]*>', '', text)
    # Remove newline characters and extra spaces
    text = text.replace('\n', ' ').replace('\r', ' ').strip()
    return text

# Define the constant prompt
prompt = ("You are given a threat report that describes a cyber incident. "
          "Any direct mentions of the threat actor group, specific campaign names, "
          "or malware names responsible have been replaced with [PLACEHOLDER]. "
          "Your task is to analyze the report and attribute the incident to a known threat actor "
          "based on the techniques, tactics, procedures (TTPs), and any other relevant information "
          "described. Please provide the name of the threat actor you believe is responsible and briefly explain your reasoning.")

# Open the CSV file in write mode
try:
    with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header row
        writer.writerow(['URL', 'Text', 'Prompt'])

        # Iterate over each item in the results
        for item in response_results:
            # Clean title and text
            title_text = f"{clean_text(item['title'])} {clean_text(item['text'])}" if 'title' in item and 'text' in item else "No title or text available"
            url = item.get('url', 'No URL available')

            # Print part of the content for debugging (first 100 characters)
            print(f"Writing: {title_text[:100]}... to the CSV file")

            # Write the URL, cleaned text, and the constant prompt to the CSV
            writer.writerow([url, title_text, prompt])

    print(f"Data has been written to {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")


Current directory: c:\Internship
Writing: Water Makara Uses Obfuscated JavaScript in Spear Phishing Campaign, Targets Brazil With Astaroth Mal... to the CSV file
Writing: Earth Simnavaz Levies Advanced Cyberattacks Against UAE and Gulf Regions Trend Micro researchers are... to the CSV file
Writing: CyberVolk: From Hacktivism to Ransomware – Researcher Exposes New Threat Cybersecurity researchers a... to the CSV file
Writing: Mind the (air) gap: GoldenJackal gooses government guardrails The GoldenJackal threat actor has been... to the CSV file
Writing: North Korean APT Stonefly Continues Extortion Attacks Against U.S. Targets North Korean cyberespiona... to the CSV file
Writing: China-linked CeranaKeeper Group Targets Southeast Asia with Data Exfiltration Attacks A new threat a... to the CSV file
Writing: FIN7 Hackers Launch Deepfake Nude Generator Sites to Spread Malware FIN7 gang is hiding malware in A... to the CSV file
Writing: Andariel Hacking Group Shifts Focus to Financial Attack

In [ ]:
import csv
import os
import re

# Function to read unique groups from the unique_groups.csv file
def read_unique_groups(file_path):
    unique_groups = set()
    try:
        with open(file_path, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            # Assuming unique groups are in the first column
            next(reader)  # Skip header if there is one
            for row in reader:
                if row:
                    unique_groups.add(row[0].strip().lower())  # Add unique group to the set (case-insensitive)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
    return unique_groups

# Function to replace text in the report based on unique groups
def replace_with_placeholder(text, unique_groups):
    original_text = text
    # Convert the text to lowercase for case-insensitive matching
    text_lower = text.lower()
    
    # Iterate over all unique groups and replace occurrences with [PLACEHOLDER]
    for group in unique_groups:
        # Use a case-insensitive regex replacement
        text = re.sub(r'\b' + re.escape(group) + r'\b', '[PLACEHOLDER]', text, flags=re.IGNORECASE)
    
    # Return the modified text and a flag indicating if any change occurred
    return text, original_text != text

# Paths to the input files
output_csv_path = 'output.csv'
unique_groups_csv_path = 'unique_groups.csv'
modified_csv_path = 'modified_output.csv'

# Read the unique groups from the CSV
unique_groups = read_unique_groups(unique_groups_csv_path)

# Open the output CSV file and read the rows
try:
    with open(output_csv_path, mode='r', newline='', encoding='utf-8') as input_file:
        reader = csv.reader(input_file)
        # Read the header
        header = next(reader)
        
        # Prepare to write the modified output
        with open(modified_csv_path, mode='w', newline='', encoding='utf-8') as output_file:
            writer = csv.writer(output_file)
            writer.writerow(header)  # Write the header to the new file

            # Iterate through each row in the original CSV
            for row in reader:
                url = row[0]
                text = row[1]
                prompt = row[2]

                # Replace any terms in text with [PLACEHOLDER]
                modified_text, changed = replace_with_placeholder(text, unique_groups)
                
                # If any changes were made, write the modified row to the new file
                if changed:
                    writer.writerow([url, modified_text, prompt])

    print(f"Modified data has been written to {modified_csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")


Modified data has been written to modified_output.csv


: 